# Statistical Analysis of RNAseq Data

This notebook contains some python code that will collate the `kallisto` outputs you generated in the previous notebook and store these as a python object called a dataframe. If you are not familiar with python, you can conceptualise a dataframe as being a spreadsheet that python uses to store and retrive data. 

You don't need to write any code here, but if you are curious, I encourage you to look at the code I have written and see if you work out what is going on.

After the dataframe has been made, there is some additional python code that passes this to different programs from software package called `pydeseq2`. These programs process the data and carry out a statistical test that compares transcript counts between the healthy and colorectal cancer patient groups to identify genes that are differentially expressed between the two groups. Finally, a table is printed that shows the names of the genes that were found to have statistically significant changes in experssion level between the two groups after adjusting for multiple comparisons.

## Further reading for those of you who want to know more about how all of this works

__________________

**Publication describing `pydeseq2`:** 

https://academic.oup.com/bioinformatics/article/39/9/btad547/7260507

___________________

**Documentation for `pydeseq2`:**   

https://pydeseq2.readthedocs.io/en/latest/

___________________

**Publication describing the statistical methods that underpin `pydeseq2`:** 

https://genomebiology.biomedcentral.com/articles/10.1186/s13059-014-0550-8

__________________

In [ ]:
import pandas as pd
import os
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

In [ ]:
cd ~339_lab-main/outputs 

In [ ]:
start_directory = './' 

In [ ]:
def find_tsv_files(start_directory):
    tsv_files = []
    for root, _, files in os.walk(start_directory):
        for file in files:
            if file.endswith('.tsv'):
                full_path = os.path.join(root, file)
                tsv_files.append(full_path)
    return tsv_files

def concatenate_tsv_files(tsv_files):
    dfs = []
    for file_path in tsv_files:
        df = pd.read_csv(file_path, sep='\t')
        df['file_path'] = file_path
        dfs.append(df)
    concatenated_df = pd.concat(dfs, ignore_index=True)
    return concatenated_df
    
def gather_and_concat(start_directory):
    tsv_files = find_tsv_files(start_directory)
    concatenated_df = concatenate_tsv_files(tsv_files)
    return concatenated_df

In [ ]:
concatenated_df = gather_and_concat(start_directory)

#make an all zeros df with the uniqe patient and gene names
row_names=concatenated_df['file_path'].unique().tolist()
column_names=concatenated_df['target_id'].unique().tolist()
count_df= pd.DataFrame(1, index=row_names, columns=column_names)

#update the empty count df with the appropriate values by iterating the concatenated df
# Initialize an empty list to store the tuples
xyz_tuples = []

# Iterate through each row in the DataFrame and get values from columns 'x', 'y', and 'z'
for row in concatenated_df.itertuples(index=False):
    x_value = getattr(row, 'file_path')
    y_value = getattr(row, 'target_id')
    z_value = getattr(row, 'tpm')
    
    # Create a tuple from the values and add it to the list
    xyz_tuples.append((x_value, y_value, z_value))

#use tuple  values to update count df
for i in xyz_tuples:
    count_df.loc[i[0]][i[1]]=int(i[2])

count_df=count_df.sort_index()

genes_to_keep = count_df.columns[count_df.sum(axis=0) >= 10]
count_df = count_df[genes_to_keep]

In [ ]:
groups = ['c','c','c','h','h','h','h','h','h','c','c','c']
metadata = pd.DataFrame(groups, index=sorted(row_names), columns=["group"])

In [ ]:
dds = DeseqDataSet(
    counts=count_df,
    metadata=metadata,
    design_factors="group",
    refit_cooks=True,
    n_cpus=1,)

dds.deseq2()
stat_res = DeseqStats(dds, n_cpus=1)
stat_res_c_vs_h = DeseqStats(dds, contrast=["group", "c", "h"],alpha=0.05, n_cpus=1,cooks_filter=True, independent_filter=True)
stat_res_c_vs_h.summary()

In [ ]:
stat_res_c_vs_h.results_df[stat_res_c_vs_h.results_df['padj']<0.05]

# What can we learn from these results?

If all has gone according to plan, the table above should contain gene names in the first row that end with `genome_9_orf00042`, `genome_9_orf00043`, `genome_9_orf00045` and  `genome_9_orf00046` All of the carryon before this is some weird side effect of the helper code I wrote, and I haven't figured out how to remedy this just yet. 

Essentially, this is saying that the genes **orf00042, orf00043, orf00045 and  orf00046** from **genome_9** are differentially expressed between the two groups we compared.

If we look at the `log2FoldChange` column, all of the values are positive, which tells us they were **upregulated** in the colorectal cancer group

All of the values in the `padj` column less than 0.0005, which tells us that after adjusting for multiple comparisons, these differences are **highly statistically significant**.

**What should we do now?** 

Well .... bcause I made up these data, I know exactly what we should do now!

In the final cell of this note book, I have added the gene sequences for orf00042, orf00043, orf00045 and  orf00046 from genome_9. We would like to see what these genes might be doing. A good starting point is to compare them to a sequence database to see if they match any genes with known function. We will use the online version of the sequence search tool **blastx** to do this.

The link below will take you to the blastx webserver. Once you are there, you can cut and paste all of the sequences into the search box and click the **blast** button at the bottom of the page. The default parameters should be fine for this.

**Link for blastx:**

https://blast.ncbi.nlm.nih.gov/Blast.cgi?LINK_LOC=blasthome&PAGE_TYPE=BlastSearch&PROGRAM=blastx

When the search has finished running, you will automatically be redirected to a page that displays the results. Using the drop down menu next to the line labelled `Results for` near the top of the page, you can select which of the sequences you want to view the results for.

**And then what?** 

That's up to you! Here are some suggestions to get you started: 

 - Look at the names of top few hits for each of the genes
 
 - Do they share anything in common?
 
 - Click on the links for some of these and see if you can find the publication(s) that relate to the hit sequence(s)
 
 - What can you learn from these publications?
 
 - Do the known functions of these genes suggest anything about the link between the microbiome and cancer risk. Remember, we know that the genes we searched were significantly upregulated in the microbiomes of patients who went on to develop colorectal cancer

```
>genome_9_orf00042 
ATGACGAAGGATGTCGCACTGATGTTCCCTGGCTCCGGTTCGCAATATGTAGGCATGGCA
CGGTGGCTGTATGAGCGTTATCCGCAGGTGCGTACTCTGTTTGATGAAGCTAGCCAGATC
ACGGAACGGGACATGGCAGCGCTGTGCCTGTCGGGTACTTTGGTACAACTTGCTGAGCCG
ACGGCAATGGCGCTGGCTATCTATACCACCAGCGTGGCCCACTTTGTCGCGTGGCAGCAG
TTTTTGGCGCAAACCGGTGTCCATGTCAACCTACGTTATATGTTGGGTCATAGTTTGGGC
GAATATGCTGCGCTGACCTGTAGTGGTGCGCTGAGCTTTTCCCAAGCACTGGCGCTGGTG
GCGATGCGTAGCCGTTTAGCCAGTGAGATCGCCCGCGAAATGGACGCGAGTACCACTATC
ATCAAGCAAGGGAATCAGGCACTGGTGGCAGCGGCGTGCGAGGTGGCGGAGCGTGAAACC
CGACAGCAGGTCGGTATTGCCTGCTTCAATTCGCCGCAACAGTTTATGTTATCCGGACAA
AACTCGGCCATTATCGCCGCCGAGCAATACCTGCTGGATCATGATCGCCAAGTCGAAGTA
GTGCCGCTGATTGGTGGTGTTCCTTACCACAGCCCACTATTGAAACCTTGCGGACAGCAA
TTGCGCAAGGCGCTGGATCGCTGCGAATGGCGACGTCCGTGCTGCCCAGTGATCTCCAAT
GTCAACGCTCAACCCTATCCCGATACGGTCACGCCAGTCCAGTGGCTAGAACAACAGCTC
TCTCAGCCAGTGCAGTGGCAGCGCTCTCTCACCTACCTGACAGGACACTTGTCACCGATC
GCGATAGAGATCGGTCCGCAAAGTGTGCTGAAAAACCTGCTGCTGGAGAATCGCTATCCA
GCACCAGTTTACGCATTTGACAATCGTCATGATCGTGCGCAACTAGCATTGGTGCTGGGG
GATAACATGGCGGTGAAGACCGATCCGGAAGCCGTGCGCCGCCAACGCATCACGCTGCTC
ACTAACGCCCTGACAGCCACACGCCATCACCGCGCCGCCGATGTGGCTGCCAGTGCGCAA
TTGAAAGAATTGCTCAGTCGTTTTTTTGAGCGCATCCAGCAGATTGAGCAGCGCGGCACA
TCCAGCGAAGAGGACATTGCTTTTTTGCATGAGCTTCTTGAACAGGGATTTCAACTCAAA
GGCAGTAGTCAGGCAGAAATCGACGCCTGCCACGCGCGGTTGGCTTCCGACAACGGCGGA
CAGGCG
>genome_9_orf00043
ATGTGCACTGAAAATTATGAGCTGGCTCAGCAAGAGGCAGTCCTGTTTGCCAAACAACAT
CTCGCCCTGGCTGCACAGAATATTGAACGTCAGCAGTTTATTGTGCCAGACATTATTTCT
TGCGTGGCGCAGGCCGGGTACTTAGGTGCGTCAATCCCCCAAAAATATGGCGGACGAGGT
TACGATTCTTATCAACTGTGCGCTTTGCATGAAGTGATGGCTGGGGTTCACGGTTCGTTA
GAAAATCTCATAACAGTGACTGGCATGGTCAGCACGCTGCTGCAACGCGTGGGCAGCGCA
GCGCAAAAAGCCCATTATTTGCCAAAGCTGGCAACGGGGGAATTGATCGGCGCGATTGCC
CTCACAGAGCCGAATATCGGCAGTGACTTAGTGAACGTGGAAACAGAATTACAGCAGGAT
GGCGACGGTTGGCGGCTGAACGGGAAAAAGAAATGGATCACGCTAGGGCAGATTGCTGAT
TTTTTTATTGTTTTAATCCACTGTGGTAATCAATTAGCGACAGTGCTTATCGATCGCAAT
ACCGACGGCTTTACTATCACTCCACTCAACGACATGTTGGGGTTACGCGGCAATATGTTG
GCAGAGCTGCATTTTAATGACTGCCGCCTGAAGGAGGATGCATTGCTCGGTCCGCTCACC
CCTGGGGTACCGCTGGCGGTGAATTTTGCGCTCAATGAAGGGCGGTTCACCACCGCTTGT
GGCAGCCTAGGATTATGTCGGGCCGCTGTGGATGTGGCAGCGCGCTACATCCGCCAGCGT
AAACAGTTCAAGCGGCGGTTATTTTCTCATGGCATTGTTCAGCACTTGTTTGCCACAATG
CTGACCCAAACGCGCAGTGCGCAACTAATGTGCTTCAGCGCGGCGGAATACCGCGAAACG
CTGCACCCGGCCATGATAAACCAAATCCTGATGGCGAAGTATGTCGCTTCCAAAGCTGCG
GTGGACGTGGCCGGGAAGGCGGTGCAACTGCTCGGTGCGAATGGTTGCCATGCCGATTAT
GCCGTCGAACGTTACTACCGCGACGCCAAAATCATGGAAATCATCGAAGGGACATCGCAA
ATTCATGAAATCCAAATTGCGATGAATTACATGATGGGGAGTGAAGCA
>genome_9_orf00045
ATGAATATGAAAAAGCAAGATATGAAAGCCGCCATTCGGGAATTTCTTTCACGCTCATTA
CGTGGGCATACGTTGAACGATGATGACGATATTTTTTCTCTCGGGCTTGTCCATTCGTTA
TTTACTGTGCAAATCATACTGTTTATAGAAAAAAATTTTCAGGTTGAGCTGGAAGTGAGT
GAGTTGAAAACAGAACAGATTGCTACCGTCAATAAAATAGTGGAGCTCATTCAGCGACAA
ACAGGCCTGGAG
>genome_9_orf00046
ATGATGAACGTGGCGGTAATAGGTGCAGGAGTAATGGGAACTGGCGTCGCTCATAACATG
GCGCAATACGGCATATCAACTAACGTTGTGGATATTTCTCAATCTCAGTTGGATAAATGC
CGACAGATGATCGAAGCGAACTTACGCTTATATAATTTTCACCCGCAGCATAAAAAAAAG
ACGCATAGCACAGCGGAGATAATGGAAAATATCCGTTTCACAACTGAGTTGGACGACATT
GTAGAATGTGATTTGGTGATTGAGAATATTACTGAGGATATAGAGAAAAAAAATGCACTA
TACACACGGATGAATACGATCTGCGGCGCGTCGACAGTGTTTGGTGTTAATACGTCCGCC
ATATCTATCACTGCGTTGTCAAAATTAATGCGACATCCGGAGAATGTAGTCGGCGTCCAT
TTTATGAACCCGGTACCGCTGATGCACACCGTCGAATTAATCCGTGGTGTACATACTGCA
GAGCGTACACTGAACATATTTCACCACTTATTTGCTCAGTTGAATAAAACCGGCATCGTG
GTCAATGATTCTCCGGGCTTTGTCACTAATCGTGCCATGATGATTTTTGTTAACGAAGCC
ATATTTATGGTGCAGGAGCAGATCGCCCGTGCTGAGGATATCGATACGTTGTTTAAAACC
TGCTTCGGGCACAAGATGGGGCCACTACAAACTGCCGATTTGATTGGCTTGGATACGATT
TTGCAATCGTTGCAGGTGCTATATGAAAGTTTTAATGATGATAAGTATCGCCCCAGCTTT
TTATTAAAGAAAATGGTCGATGCAGGGTATTTGGGCGTGAAATCAGGGCAGGGGTTTTAT
CGCTATCAACAGACGTACGCCGAGCAG
```